In [1]:
!pip install pandas
!pip install pypdf
!pip install pypdf2
!pip install cryptography
!pip install pikepdf

  Using cached pypdf-5.4.0-py3-none-any.whl.metadata (7.3 kB)
Using cached pypdf-5.4.0-py3-none-any.whl (302 kB)


In [79]:
import os
import shutil
import pikepdf
import re
import pandas as pd
from PyPDF2 import PdfReader

# Caminhos das pastas
source_folder = "./maps/encrypted/"
processed_folder = os.path.join(source_folder, "processed")
decrypted_folder = "./maps/decrypted/"
decrypted_processed_folder = os.path.join(decrypted_folder, "processed")

# Função para verificar e criar a hierarquia de pastas
def ensure_folders_exist():
    folders = [source_folder, processed_folder, decrypted_folder, decrypted_processed_folder]
    for folder in folders:
        if not os.path.exists(folder):
            os.makedirs(folder)
            print(f"Pasta criada: {folder}")
        else:
            print(f"Pasta já existe: {folder}")

# Cria a hierarquia de pastas
ensure_folders_exist()


# Busca o primeiro arquivo PDF na pasta maps
pdf_files = [f for f in os.listdir(source_folder) if f.endswith('.pdf')]

if not pdf_files:
    print("Nenhum arquivo PDF encontrado na pasta maps.")
else:
    encrypted_pdf_path = os.path.join(source_folder, pdf_files[0])
    decrypted_pdf_path = os.path.join(decrypted_folder, f"decrypted_{pdf_files[0]}")

    try:
        # Tenta descriptografar o PDF
        with pikepdf.open(encrypted_pdf_path) as pdf:
            pdf.save(decrypted_pdf_path)
        print(f"PDF desbloqueado com sucesso! Salvo em: {decrypted_pdf_path}")

        # Move o PDF criptografado para a pasta processed
        shutil.move(encrypted_pdf_path, os.path.join(processed_folder, pdf_files[0]))
        print(f"PDF original criptografado movido para: {processed_folder}")

    except pikepdf.PasswordError:
        print(f"O PDF '{pdf_files[0]}' está protegido por senha e não pode ser desbloqueado sem ela.")
    except FileNotFoundError as e:
        print(f"Erro: Arquivo não encontrado. Verifique o caminho. {e}")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")




# Paths das pastas
input_folder = "./maps/decrypted"
processed_folder = "maps/decrypted/processed"

# Verificar se a pasta de processados existe, caso contrário, criá-la
os.makedirs(processed_folder, exist_ok=True)

# Obter a lista de PDFs na pasta de entrada
pdf_files = [file for file in os.listdir(input_folder) if file.lower().endswith(".pdf")]

if pdf_files:
    # Processar cada PDF na pasta
    for file_name in pdf_files:
        input_pdf_path = os.path.join(input_folder, file_name)
        processed_pdf_path = os.path.join(processed_folder, file_name)
        
        try:
            # Inicializar o texto completo para o PDF atual
            full_text = ""
            
            # Ler o PDF
            reader = PdfReader(input_pdf_path)
            
            # Iterar sobre as páginas e adicionar os delimitadores
            for idx, page in enumerate(reader.pages):
                full_text += f"------------------ PAGINA {idx} ----------------\n"
                full_text += page.extract_text()
            
            # Dividir o texto baseado nos delimitadores
            paginas = re.split(r"------------------ PAGINA \d+ ----------------", full_text)
            paginas = [pagina.strip() for pagina in paginas if pagina.strip()]
            
            # Exibir a quantidade de páginas úteis detectadas
            print(f"\nTotal de páginas detectadas em '{file_name}': {len(paginas)}\n")
            
            # Criar dicionário para armazenar o conteúdo das páginas
            texto_paginas = {}
            for idx, pagina in enumerate(paginas, start=1):
                texto_paginas[f"texto_pagina{idx}"] = pagina
            
        except Exception as e:
            print(f"Ocorreu um erro ao processar '{file_name}': {e}")
        finally:
            # Mover o PDF processado para a pasta de processados
            shutil.move(input_pdf_path, processed_pdf_path)
            print(f"PDF '{file_name}' movido para a pasta de processados: {processed_pdf_path}")
else:
    print("Nenhum PDF encontrado na pasta de entrada.")



# Regexes para capturar os elementos de interesse
regexes = {
    'nome': re.compile(r'Nome:\s+(.+)', re.MULTILINE),
    'nif': re.compile(r'Nº de Identificação:\s+(\d+)'),
    'mes_mapa': re.compile(r'Responsabilidades de crédito referentes a\s+(.+)'),
    'instituicao': re.compile(r'Informação comunicada pela instituição:\s+(.+)'),
    'total_em_divida': re.compile(r"Montantes\s+Total em dívida\s+do qual, em incumprimento\s+([\d\s,]+) €"),
    'litigio': re.compile(r'Em litígio judicial\s+(Sim|Não)'),
    'abatido_ativo': re.compile(r'Abatido ao ativo\s+([\d\s,.]+) €'),
    'garantias': re.compile(r"Tipo\s+Valor\s+Número\s+\d+\s+([\d\s,.]+) €"),
    'num_devedores': re.compile(r"Nº devedores no contrato\s+(\d+)"),
    'prod_financeiro': re.compile(r"Produto financeiro\s+(.+?)\s+Tipo de responsabilidade"),
    'dat_inicio': re.compile(r"Início\s+(\d{4}-\d{2}-\d{2})"),
    'dat_fim': re.compile(r"Fim\s+(\d{4}-\d{2}-\d{2})\s+Em litígio judicial"),
    'entrada_incumpr': re.compile(r"Entrada incumpr\.\s+(\d{4}-\d{2}-\d{2})\s+Tipo")
}

# Função para aplicar regex em texto
def get_feature(text, regex_string):
    match = regexes[regex_string].search(text)
    return match.group(1).strip() if match else None
 
# Inicializar lista para armazenar os dados
data = []

# Iterar sobre as páginas do dicionário texto_paginas
for page_number, page_text in texto_paginas.items():
    # Extrair informações de cada página usando regex
    row = {
        'pagina_pdf': page_number,
        'nome': get_feature(page_text, 'nome'),
        'nif': get_feature(page_text, 'nif'),
        'mes_mapa': get_feature(page_text, 'mes_mapa'),
        'instituicao': get_feature(page_text, 'instituicao'),
        'divida': get_feature(page_text, 'total_em_divida'),
        'litigio': get_feature(page_text, 'litigio'),
        'parcela': get_feature(page_text, 'abatido_ativo'),
        'garantias': get_feature(page_text, 'garantias'),
        'num_devedores': get_feature(page_text, 'num_devedores'),
        'prod_financeiro': get_feature(page_text, 'prod_financeiro'),
        'entrada_incumpr': get_feature(page_text,'entrada_incumpr'),
        'dat_inicio': get_feature(page_text, 'dat_inicio'),
        'dat_fim': get_feature(page_text, 'dat_fim')
    }
    # Adicionar os dados ao conjunto
    data.append(row)

# Criar o DataFrame
df = pd.DataFrame(data)
df.iloc[:-1]

Pasta já existe: ./maps/encrypted/
Pasta já existe: ./maps/encrypted/processed
Pasta já existe: ./maps/decrypted/
Pasta já existe: ./maps/decrypted/processed
PDF desbloqueado com sucesso! Salvo em: ./maps/decrypted/decrypted_f5faf0a3-6ede-4761-9993-f498dc341586.pdf
PDF original criptografado movido para: ./maps/encrypted/processed

Total de páginas detectadas em 'decrypted_f5faf0a3-6ede-4761-9993-f498dc341586.pdf': 3

PDF 'decrypted_f5faf0a3-6ede-4761-9993-f498dc341586.pdf' movido para a pasta de processados: maps/decrypted/processed/decrypted_f5faf0a3-6ede-4761-9993-f498dc341586.pdf


,pagina_pdf,nome,nif,mes_mapa,instituicao,divida,litigio,parcela,garantias,num_devedores,prod_financeiro,entrada_incumpr,dat_inicio,dat_fim
0,texto_pagina1,SARA CRISTINA PEREIRA FRAZÃO,245914331,agosto de 2023,"CAIXA DE CRÉDITO AGRÍCOLA MÚTUO DO RIBATEJO SUL, CRL (5470)","924,00",Não,"0,00",None,1,Cartão de crédito - com período de free-float,None,2020-01-17,9999-12-31
1,texto_pagina2,SARA CRISTINA PEREIRA FRAZÃO,245914331,agosto de 2023,"321CRÉDITO, INSTITUIÇÃO FINANCEIRA DE CRÉDITO S.A. (0305)","4 132,62",Sim,"0,00",None,1,Crédito automóvel (excluíndo locações financeiras),2023-02-24,2020-09-01,2023-02-24


In [87]:
import os
import shutil
import pikepdf
import re
import pandas as pd
from PyPDF2 import PdfReader

# Caminhos das pastas
source_folder = "./maps/encrypted/"
processed_folder = os.path.join(source_folder, "processed")
decrypted_folder = "./maps/decrypted/"
decrypted_processed_folder = os.path.join(decrypted_folder, "processed")

def ensure_folders_exist():
    folders = [source_folder, processed_folder, decrypted_folder, decrypted_processed_folder]
    for folder in folders:
        os.makedirs(folder, exist_ok=True)

ensure_folders_exist()

# Descriptografar PDF
pdf_files = [f for f in os.listdir(source_folder) if f.endswith('.pdf')]
if not pdf_files:
    print("Nenhum PDF encontrado.")
else:
    encrypted_pdf_path = os.path.join(source_folder, pdf_files[0])
    decrypted_pdf_path = os.path.join(decrypted_folder, f"decrypted_{pdf_files[0]}")
    try:
        with pikepdf.open(encrypted_pdf_path) as pdf:
            pdf.save(decrypted_pdf_path)
        print(f"PDF desbloqueado com sucesso: {decrypted_pdf_path}")
        shutil.move(encrypted_pdf_path, os.path.join(processed_folder, pdf_files[0]))
    except Exception as e:
        print(f"Erro ao desbloquear: {e}")

# Processamento do PDF
input_folder = "./maps/decrypted"
processed_folder = "./maps/decrypted/processed"
os.makedirs(processed_folder, exist_ok=True)
pdf_files = [file for file in os.listdir(input_folder) if file.lower().endswith(".pdf")]

texto_blocos = {}
if pdf_files:
    for file_name in pdf_files:
        input_pdf_path = os.path.join(input_folder, file_name)
        processed_pdf_path = os.path.join(processed_folder, file_name)
        try:
            full_text = ""
            reader = PdfReader(input_pdf_path)
            for idx, page in enumerate(reader.pages):
                full_text += page.extract_text() + "\n"

            print(f"\nPDF: {file_name} | Total de páginas: {len(reader.pages)}")

            # Separar por instituições
            instituicoes = re.split(
                r"(Informação comunicada pela instituição:.+?(?=Informação comunicada pela instituição:|$))",
                full_text,
                flags=re.DOTALL
            )

            idx_geral = 1
            for inst_text in instituicoes:
                inst_match = re.search(r"Informação comunicada pela instituição:\s+(.+)", inst_text)
                nome_inst = inst_match.group(1).strip() if inst_match else "NÃO IDENTIFICADA"

                blocos = re.findall(
                    r"(Montantes.*?Produto financeiro.+?)(?=Montantes|Informação comunicada pela instituição:|$)",
                    inst_text,
                    flags=re.DOTALL
                )

                for bloco in blocos:
                    bloco_completo = f"Informação comunicada pela instituição: {nome_inst}\n{bloco.strip()}"
                    texto_blocos[f"produto_{idx_geral}"] = bloco_completo
                    idx_geral += 1

            print(f"Total de produtos detectados em '{file_name}': {len(texto_blocos)}")

        except Exception as e:
            print(f"Erro ao processar '{file_name}': {e}")
        finally:
            shutil.move(input_pdf_path, processed_pdf_path)
            print(f"PDF '{file_name}' movido para: {processed_pdf_path}")
else:
    print("Nenhum PDF encontrado na pasta de entrada.")

# Regexes
regexes = {
    'nome': re.compile(r'Nome:\s+(.+)', re.MULTILINE),
    'nif': re.compile(r'Nº de Identificação:\s+(\d+)'),
    'mes_mapa': re.compile(r'Responsabilidades de crédito referentes a\s+(.+)'),
    'instituicao': re.compile(r'Informação comunicada pela instituição:\s+(.+)'),
    'total_em_divida': re.compile(r"Total em dívida\s+do qual, em incumprimento\s+([\d\s,]+) €"),
    'litigio': re.compile(r'Em litígio judicial\s+(Sim|Não)'),
    'abatido_ativo': re.compile(r'Abatido ao ativo\s+([\d\s,.]+) €'),
    'garantias': re.compile(r"Tipo\s+Valor\s+Número[\s\S]+?([\d\s,.]+) €"),
    'num_devedores': re.compile(r"Nº devedores no contrato\s+(\d+)"),
    'prod_financeiro': re.compile(r"Produto financeiro\s+(.+?)\s+Tipo de responsabilidade"),
    'dat_inicio': re.compile(r"Início\s+(\d{4}-\d{2}-\d{2})"),
    'dat_fim': re.compile(r"Fim\s+(\d{4}-\d{2}-\d{2})"),
    'entrada_incumpr': re.compile(r"Entrada incumpr\.\s+(\d{4}-\d{2}-\d{2})")
}

def get_feature(text, regex_string):
    match = regexes[regex_string].search(text)
    return match.group(1).strip() if match else None

# Construção do DataFrame
data = []
for bloco_id, bloco_text in texto_blocos.items():
    row = {
        'bloco_id': bloco_id,
        'nome': get_feature(bloco_text, 'nome'),
        'nif': get_feature(bloco_text, 'nif'),
        'mes_mapa': get_feature(bloco_text, 'mes_mapa'),
        'instituicao': get_feature(bloco_text, 'instituicao'),
        'divida': get_feature(bloco_text, 'total_em_divida'),
        'litigio': get_feature(bloco_text, 'litigio'),
        'parcela': get_feature(bloco_text, 'abatido_ativo'),
        'garantias': get_feature(bloco_text, 'garantias'),
        'num_devedores': get_feature(bloco_text, 'num_devedores'),
        'prod_financeiro': get_feature(bloco_text, 'prod_financeiro'),
        'entrada_incumpr': get_feature(bloco_text, 'entrada_incumpr'),
        'dat_inicio': get_feature(bloco_text, 'dat_inicio'),
        'dat_fim': get_feature(bloco_text, 'dat_fim')
    }
    data.append(row)

df = pd.DataFrame(data)
df

PDF desbloqueado com sucesso: ./maps/decrypted/decrypted_ea714444-6ea1-4e90-8adf-d76cf755aa48.pdf

PDF: decrypted_ea714444-6ea1-4e90-8adf-d76cf755aa48.pdf | Total de páginas: 4
Total de produtos detectados em 'decrypted_ea714444-6ea1-4e90-8adf-d76cf755aa48.pdf': 5
PDF 'decrypted_ea714444-6ea1-4e90-8adf-d76cf755aa48.pdf' movido para: ./maps/decrypted/processed/decrypted_ea714444-6ea1-4e90-8adf-d76cf755aa48.pdf


,bloco_id,nome,nif,mes_mapa,instituicao,divida,litigio,parcela,garantias,num_devedores,prod_financeiro,entrada_incumpr,dat_inicio,dat_fim
0,produto_1,MARIA JOÃO DE OLIVEIRA FERNANDES,186707053,maio de 2023,"BANCO COMERCIAL PORTUGUÊS, SA (0033)","534,36",Não,"0,00","0,00",1,Cartão de crédito - com período de free-float,None,2005-05-02,9999-12-31
1,produto_2,None,None,None,COFIDIS (0921),"0,00",Não,"0,00","0,00",1,Crédito renovável - Linha de crédito,None,2005-05-04,9999-12-31
2,produto_3,MARIA JOÃO DE OLIVEIRA FERNANDES,186707053,maio de 2023,COFIDIS (0921),"0,00",Não,"0,00","0,00",1,Crédito renovável - Linha de crédito,None,1999-09-20,9999-12-31
3,produto_4,None,None,None,"WIZINK BANK, S.A.U. - SUCURSAL EM PORTUGAL (0272)","12 473,39",Não,"0,00","0,00",1,Cartão de crédito - sem período de free-float,None,2008-01-29,9999-12-31
4,produto_5,MARIA JOÃO DE OLIVEIRA FERNANDES,186707053,maio de 2023,"WIZINK BANK, S.A.U. - SUCURSAL EM PORTUGAL (0272)","4 477,87",Não,"97,19","97,19",1,Crédito pessoal,None,2020-05-27,2028-06-01


In [88]:
df

,bloco_id,nome,nif,mes_mapa,instituicao,divida,litigio,parcela,garantias,num_devedores,prod_financeiro,entrada_incumpr,dat_inicio,dat_fim
0,produto_1,MARIA JOÃO DE OLIVEIRA FERNANDES,186707053,maio de 2023,"BANCO COMERCIAL PORTUGUÊS, SA (0033)","534,36",Não,"0,00","0,00",1,Cartão de crédito - com período de free-float,None,2005-05-02,9999-12-31
1,produto_2,None,None,None,COFIDIS (0921),"0,00",Não,"0,00","0,00",1,Crédito renovável - Linha de crédito,None,2005-05-04,9999-12-31
2,produto_3,MARIA JOÃO DE OLIVEIRA FERNANDES,186707053,maio de 2023,COFIDIS (0921),"0,00",Não,"0,00","0,00",1,Crédito renovável - Linha de crédito,None,1999-09-20,9999-12-31
3,produto_4,None,None,None,"WIZINK BANK, S.A.U. - SUCURSAL EM PORTUGAL (0272)","12 473,39",Não,"0,00","0,00",1,Cartão de crédito - sem período de free-float,None,2008-01-29,9999-12-31
4,produto_5,MARIA JOÃO DE OLIVEIRA FERNANDES,186707053,maio de 2023,"WIZINK BANK, S.A.U. - SUCURSAL EM PORTUGAL (0272)","4 477,87",Não,"97,19","97,19",1,Crédito pessoal,None,2020-05-27,2028-06-01


In [9]:
print(full_text)

------------------ PAGINA 0 ----------------
Informação comunicada pela instituição: ONEY BANK - SUCURSAL EM PORTUGAL  (0881)
Montantes
Total em dívida
do qual, em incumprimento
229,56 €
0,00 €
Nº devedores no contrato
1
Início
2021-01-30
Tipo de negociação
Totalmente nova
Produto financeiro
Cartão de crédito - com período de free-float
Tipo de responsabilidade
Devedor
Fim
9999-12-31
Em litígio judicial
Não
Vencido
0,00 €
Entrada incumpr.
Não Aplicável
Tipo 
Valor
Número
-
-
-
Garantias
Abatido ao ativo
0,00 €
0,00 €
Potencial
520,65 €
Prestação 
Periodicidade
Mensal
Montantes
Total em dívida
do qual, em incumprimento
0,00 €
0,00 €
Nº devedores no contrato
1
Início
2023-03-13
Tipo de negociação
Totalmente nova
Produto financeiro
Cartão de crédito - com período de free-float
Tipo de responsabilidade
Devedor
Fim
9999-12-31
Em litígio judicial
Não
Vencido
0,00 €
Entrada incumpr.
Não Aplicável
Tipo 
Valor
Número
-
-
-
Garantias
Abatido ao ativo
0,00 €
0,00 €
Potencial
800,00 €
Prestação 
P